In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up gnupg-agent (2.1.15-1ubuntu8.1) ...
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up gnupg (2.1.15-1ubuntu8.1) 

In [2]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
path = "drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch"
os.chdir(path)

In [3]:
!pwd

/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch


pytorch関連のインストール

In [6]:
!pip3 install torch torchvision torchtext

    100% |████████████████████████████████| 484.0MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5cc42000 @  0x7f0a5c7c51c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 15.7MB/s 
    100% |████████████████████████████████| 51kB 12.6MB/s 
    100% |████████████████████████████████| 2.0MB 14.4MB/s 
    100% |████████████████████████████████| 51kB 12.4MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:


      Successfully uninstalled Pillow-4.0.0


In [0]:
import os
import sys
import time
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

from model import SNLIClassifier
from util import get_args, makedirs

In [14]:
sys.argv = [sys.argv[0],
            '--epochs', '10', '--d_embed', '300', # '--d_proj', '100', 
            '--dp_ratio', '0.5', '--n_layers', '2',
            '--d_hidden', '300', # '--train_embed',
            '--no-bidirectional', '--no-projection',
            '--word_vectors', 'glove.6B.300d', 
            '--vector_cache', '../input/vector_cache']
args = get_args()
args
# !python train.py --epochs 5 --d_embed 300 --gpu -1 --word_vectors glove.6B.300d

Namespace(batch_size=128, birnn=False, d_embed=300, d_hidden=300, d_proj=300, dev_every=1000, dp_ratio=0.5, epochs=10, fix_emb=True, gpu=0, log_every=50, lower=True, lr=0.001, n_layers=2, projection=False, resume_snapshot='', save_every=1000, save_path='results', vector_cache='../input/vector_cache', word_vectors='glove.6B.300d')

In [0]:
torch.cuda.set_device(args.gpu)

In [0]:
inputs = data.Field(lower=args.lower)
answers = data.Field(sequential=False, use_vocab=False)
train, dev, test = datasets.SNLI.splits(inputs, answers, root="../input")

In [0]:
inputs.build_vocab(train, dev, test)

inputs.vocab.load_vectors(args.word_vectors, cache=args.vector_cache)
# if args.word_vectors:
#     if os.path.isfile(args.vector_cache):
#         inputs.vocab.vectors = torch.load(args.vector_cache)
#     else:
#         inputs.vocab.load_vectors(args.word_vectors)
#         makedirs(os.path.dirname(args.vector_cache))
#         torch.save(inputs.vocab.vectors, args.vector_cache)

answers.build_vocab(train)

In [0]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)

In [26]:
answers.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0, 'contradiction': 2, 'entailment': 1, 'neutral': 3})

In [0]:
config = args
config.n_embed = len(inputs.vocab)
config.d_out = len(answers.vocab)
config.n_cells = config.n_layers

# double the number of cells for bidirectional networks
if config.birnn:
    config.n_cells *= 2
    
criterion = nn.CrossEntropyLoss()

In [0]:
if args.resume_snapshot:
    model = torch.load(args.resume_snapshot, map_location=lambda storage, locatoin: storage.cuda(args.gpu))
else:
    model = SNLIClassifier(config)
    if args.word_vectors:
        model.embed.weight.data.copy_(inputs.vocab.vectors)
        model.cuda(args.gpu)

opt = O.Adam(model.parameters(), lr=args.lr)

In [22]:
iterations = 0
start = time.time()
best_dev_acc = -1
train_iter.repeat = False
header = '  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy'
dev_log_template = ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{:8.6f},{:12.4f},{:12.4f}'.split(','))
log_template =     ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{},{:12.4f},{}'.split(','))
makedirs(args.save_path)
print(header)

for epoch in range(args.epochs):
    train_iter.init_epoch()
    n_correct, n_total = 0, 0
    for batch_idx, batch in enumerate(train_iter):

        # switch model to training mode, clear gradient accumulators
        model.train(); opt.zero_grad()

        iterations += 1

        # forward pass
        answer = model(batch)

        # calculate accuracy of predictions in the current batch
        n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
        n_total += batch.batch_size
        train_acc = 100. * n_correct/n_total

        # calculate loss of the network output with respect to training labels
        loss = criterion(answer, batch.label)

        # backpropagate and update optimizer learning rate
        loss.backward(); opt.step()

        # checkpoint model periodically
        if iterations % args.save_every == 0:
            snapshot_prefix = os.path.join(args.save_path, 'snapshot')
            snapshot_path = snapshot_prefix + '_acc_{:.4f}_loss_{:.6f}_iter_{}_model.pt'.format(train_acc, loss.item(), iterations)
            torch.save(model, snapshot_path)
            for f in glob.glob(snapshot_prefix + '*'):
                if f != snapshot_path:
                    os.remove(f)

        # evaluate performance on validation set periodically
        if iterations % args.dev_every == 0:

            # switch model to evaluation mode
            model.eval(); dev_iter.init_epoch()

            # calculate accuracy on validation set
            n_dev_correct, dev_loss = 0, 0
            with torch.no_grad():
                for dev_batch_idx, dev_batch in enumerate(dev_iter):
                     answer = model(dev_batch)
                     n_dev_correct += (torch.max(answer, 1)[1].view(dev_batch.label.size()) == dev_batch.label).sum().item()
                     dev_loss = criterion(answer, dev_batch.label)
            dev_acc = 100. * n_dev_correct / len(dev)

            print(dev_log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), dev_loss.item(), train_acc, dev_acc))

            # update best valiation set accuracy
            if dev_acc > best_dev_acc:

                # found a model with better validation set accuracy

                best_dev_acc = dev_acc
                snapshot_prefix = os.path.join(args.save_path, 'best_snapshot')
                snapshot_path = snapshot_prefix + '_devacc_{}_devloss_{}__iter_{}_model.pt'.format(dev_acc, dev_loss.item(), iterations)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                for f in glob.glob(snapshot_prefix + '*'):
                    if f != snapshot_path:
                        os.remove(f)

        elif iterations % args.log_every == 0:

            # print progress message
            print(log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), ' '*8, n_correct/n_total*100, ' '*12))




  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy
     5     0        50    50/4292        1% 1.205007               35.7031             
     9     0       100   100/4292        2% 1.050765               39.5469             
    12     0       150   150/4292        3% 1.085025               41.2552             
    15     0       200   200/4292        5% 1.008855               42.7422             
    18     0       250   250/4292        6% 0.952761               43.8188             
    21     0       300   300/4292        7% 1.026032               44.6172             
    24     0       350   350/4292        8% 1.010507               45.2522             
    27     0       400   400/4292        9% 1.018619               45.8535             
    30     0       450   450/4292       10% 0.923589               46.6319             
    33     0       500   500/4292       12% 1.008312               47.3031             
    37     0       550   550/42

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


    80     0      1000  1000/4292       23% 0.965674 1.095278      51.7508      44.8283
    96     0      1050  1050/4292       24% 0.900155               52.0365             
    99     0      1100  1100/4292       26% 0.874694               52.3047             
   102     0      1150  1150/4292       27% 0.941088               52.5571             
   105     0      1200  1200/4292       28% 0.918072               52.7949             
   108     0      1250  1250/4292       29% 0.890997               53.0181             
   111     0      1300  1300/4292       30% 0.833695               53.2224             
   114     0      1350  1350/4292       31% 0.907167               53.3924             
   117     0      1400  1400/4292       33% 0.860202               53.5681             
   120     0      1450  1450/4292       34% 0.851732               53.7527             
   123     0      1500  1500/4292       35% 0.868774               53.9271             
   126     0      1550  1550/429

   280     0      3350  3350/4292       78% 0.859386               57.8300             
   283     0      3400  3400/4292       79% 0.893050               57.9035             
   286     0      3450  3450/4292       80% 0.783868               57.9726             
   289     0      3500  3500/4292       82% 0.860650               58.0435             
   292     0      3550  3550/4292       83% 0.707422               58.1224             
   295     0      3600  3600/4292       84% 0.768325               58.2014             
   298     0      3650  3650/4292       85% 0.721168               58.2622             
   301     0      3700  3700/4292       86% 0.919090               58.3404             
   304     0      3750  3750/4292       87% 0.798066               58.4150             
   307     0      3800  3800/4292       89% 0.753990               58.4671             
   311     0      3850  3850/4292       90% 0.869735               58.5294             
   314     0      3900  3900/429

   469     1      5700  1408/4292       33% 0.871729               65.0146             
   472     1      5750  1458/4292       34% 0.734570               65.0206             
   475     1      5800  1508/4292       35% 0.843190               65.0686             
   478     1      5850  1558/4292       36% 0.919327               65.0579             
   481     1      5900  1608/4292       37% 0.804819               65.0653             
   485     1      5950  1658/4292       39% 0.808262               65.0761             
   503     1      6000  1708/4292       40% 0.754204 1.088112      65.0551      46.9722
   506     1      6050  1758/4292       41% 0.782723               65.0602             
   509     1      6100  1808/4292       42% 0.814925               65.0447             
   512     1      6150  1858/4292       43% 0.763954               65.0157             
   515     1      6200  1908/4292       44% 0.823815               65.0137             
   518     1      6250  1958/429

   659     1      8050  3758/4292       88% 0.794548               65.5551             
   662     1      8100  3808/4292       89% 0.803797               65.5710             
   665     1      8150  3858/4292       90% 0.814640               65.5861             
   668     1      8200  3908/4292       91% 0.695932               65.6000             
   672     1      8250  3958/4292       92% 0.896386               65.6157             
   675     1      8300  4008/4292       93% 0.698804               65.6250             
   678     1      8350  4058/4292       95% 0.792858               65.6310             
   681     1      8400  4108/4292       96% 0.727315               65.6364             
   684     1      8450  4158/4292       97% 0.717422               65.6485             
   687     1      8500  4208/4292       98% 0.705622               65.6753             
   690     1      8550  4258/4292       99% 0.794242               65.6861             
   695     2      8600    16/429

   837     2     10400  1816/4292       42% 0.727212               68.0501             
   840     2     10450  1866/4292       43% 0.737566               68.0646             
   843     2     10500  1916/4292       45% 0.699483               68.0719             
   846     2     10550  1966/4292       46% 0.728310               68.0808             
   849     2     10600  2016/4292       47% 0.756892               68.0590             
   852     2     10650  2066/4292       48% 0.856493               68.0485             
   855     2     10700  2116/4292       49% 0.814746               68.0526             
   858     2     10750  2166/4292       50% 0.685603               68.0903             
   861     2     10800  2216/4292       52% 0.661667               68.0865             
   864     2     10850  2266/4292       53% 0.768069               68.0867             
   868     2     10900  2316/4292       54% 0.842363               68.0821             
   871     2     10950  2366/429

KeyboardInterrupt: ignored

モデルを読み込んでテストデータで評価

In [23]:
path_model_eval = "results/best_snapshot_devacc_77.52489331436699_devloss_0.7100985050201416__iter_31000_model.pt"
model_eval = torch.load(path_model_eval, 
                        map_location=lambda storage, 
                        locatoin: storage.cuda(args.gpu))

def evaluate(model, data_iter, data_size):
    log_template = '{:>8.6f}, {:8.6f}, {:12.4f}%'
    model.eval(); data_iter.init_epoch()
  
    # calculate accuracy on validation set
    n_correct, loss = 0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_iter):
            answer = model(batch)
            n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
            loss = criterion(answer, batch.label)
    acc = 100. * n_correct / data_size

    print(log_template.format(loss.item(), loss.item(), acc))
        
evaluate(model_eval, test_iter, len(test))

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch/model.py:34: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  outputs, (ht, ct) = self.rnn(inputs, (h0, c0))


0.663549, 0.663549,      76.4047%


google driveとの同期プロセスも完全にkillする

In [0]:
# !kill -9 -1